In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma beautifulsoup4

In [11]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "conversational-rag"
os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

········
········


In [3]:
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

········


In [12]:

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

In [13]:

response = llm.invoke("Whats one good song of daft punk you recommend to listen while programming?")
print(response)


content='One great Daft Punk song to listen to while programming is "Harder, Better, Faster, Stronger" from their 2001 album "Discovery."\n\nThis track is an excellent choice for several reasons:\n\n1. It has a steady, driving beat that can help maintain focus and rhythm while coding.\n\n2. The lyrics, though repetitive, emphasize improvement and efficiency, which can be motivating for programmers.\n\n3. The song\'s electronic sound fits well with the technological nature of programming.\n\n4. It\'s upbeat and energetic, which can help maintain a positive mood during long coding sessions.\n\n5. The track isn\'t too distracting or complex, allowing you to concentrate on your work while still enjoying the music.\n\nOf course, music preference is subjective, and Daft Punk has many other great tracks that might suit your taste better, such as "Around the World," "One More Time," or "Technologic." Ultimately, the best song is the one that helps you focus and enjoy your programming sessions.

In [14]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding = OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)




In [18]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response)

{'input': 'What is Task Decomposition?', 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the pr

In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [21]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

There are three common ways of performing Task Decomposition. First, using language models with simple prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ?". Second, employing task-specific instructions, such as "Write a story outline" for novel writing. Third, incorporating human inputs to break down the task. These methods help in systematically dividing complex problems into more manageable subtasks.


In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [23]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition is a technique for breaking down complex tasks into smaller, more manageable steps. It is commonly used in AI systems to help models tackle complicated problems more effectively. Task decomposition can be accomplished through methods like Chain of Thought prompting, where a model is instructed to "think step by step", or Tree of Thoughts, which explores multiple reasoning possibilities at each step. The goal is to transform big tasks into multiple simpler subtasks that are easier for AI models to process and solve.'

In [24]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'There are three common ways of performing task decomposition:\n\n1. Using language models with simple prompts: This involves asking an AI model to break down a task using prompts like "Steps for XYZ:" or "What are the subgoals for achieving XYZ?"\n\n2. Utilizing task-specific instructions: For example, when writing a novel, the instruction might be "Write a story outline."\n\n3. Incorporating human input: This method involves people directly providing guidance on how to break down complex tasks.\n\nThese approaches help in systematically dividing large, complex tasks into more manageable subtasks, making problem-solving more efficient and effective for AI systems.'

In [25]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is a technique for breaking down complex tasks into smaller, more manageable steps. It is commonly used in AI systems to help models tackle complicated problems more effectively. Task decomposition can be accomplished through methods like Chain of Thought prompting, where a model is instructed to "think step by step", or Tree of Thoughts, which explores multiple reasoning possibilities at each step. The goal is to transform big tasks into multiple simpler subtasks that are easier for AI models to process and solve.

User: What are common ways of doing it?

AI: There are three common ways of performing task decomposition:

1. Using language models with simple prompts: This involves asking an AI model to break down a task using prompts like "Steps for XYZ:" or "What are the subgoals for achieving XYZ?"

2. Utilizing task-specific instructions: For example, when writing a novel, the instruction might be "Write a story outline."

3.

In [26]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [27]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It involves transforming big tasks into multiple manageable tasks to facilitate problem-solving. This process can be done using prompting techniques like Chain of Thought or Tree of Thoughts, task-specific instructions, or human inputs.'

In [28]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Task decomposition can be achieved through various methods such as using prompting techniques like Chain of Thought or Tree of Thoughts, providing task-specific instructions tailored to the nature of the task, or incorporating human inputs to break down complex tasks into smaller, more manageable steps. These approaches help in enhancing model performance on complex tasks by enabling a step-by-step thinking process and facilitating problem-solving.'

In [29]:
#Agents
#retriever tool

from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(retriever,
                            "blog_post_retriever",
                            "Searches and returns exerpts from Autonomous Agents blog post.",)
tools = [tool]

In [30]:
tool.invoke("What is Task Decomposition?")

'Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\n\nFig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by st

In [31]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(llm, tools)


In [32]:
query = "What is Task Decomposition?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Task decomposition is a problem-solving strategy that involves breaking down a complex task or problem into smaller, more manageable subtasks. By decomposing a task into smaller components, it becomes easier to understand, analyze, and solve the overall problem. This approach allows individuals to focus on one specific aspect of the task at a time, leading to more efficient problem-solving and decision-making.\n\nWould you like me to provide more information on task decomposition from the Autonomous Agents blog post?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 69, 'total_tokens': 162}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d3bb09d3-278d-4919-92e4-67b70d3a2899-0', usage_metadata={'input_tokens': 69, 'output_tokens': 93, 'total_tokens': 162})]}}
----


In [33]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [35]:
config = {"configurable": {"thread_id": "abc123"}}

for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm Madhu")]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Madhu! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 92, 'total_tokens': 104}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3d54adf4-6099-4e12-87b6-2dc7a4d94042-0', usage_metadata={'input_tokens': 92, 'output_tokens': 12, 'total_tokens': 104})]}}
----


In [36]:
query = "What according to the blog post are common ways of doing it? redo the search"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dkzHocBj6D61MGw7IkMxcEDI', 'function': {'arguments': '{"query":"common ways of doing it"}', 'name': 'blog_post_retriever'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 127, 'total_tokens': 148}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dd819b0f-5d21-4410-8528-ec4303119ee3-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'common ways of doing it'}, 'id': 'call_dkzHocBj6D61MGw7IkMxcEDI', 'type': 'tool_call'}], usage_metadata={'input_tokens': 127, 'output_tokens': 21, 'total_tokens': 148})]}}
----
{'tools': {'messages': [ToolMessage(content='}\n]\nChallenges#\nAfter going through key ideas and demos of building LLM-centered agents, I start to see a couple common limitations:\n\n}\n]\nChallenges#\nAfter going through key ideas and